# Laboratorio 02: Representación Numérica, Errores y Estabilidad.

---
### Profesor: Daniel Ruiz Mejía
### Nombre: Dana Ines Romero Bustos
*Métodos computacionales 2025-I*

---

# 1.
Cree una función llamada `myint` pero que a partir de un número binario con base de 16 bits encuentre el entero correspondiente. Compare su resultado con `int(0b1000011100001)`

In [13]:
import math as mt
import numpy as np
import matplotlib.pyplot as plt

In [14]:
def myint(binario):
    '''
    Esta funcion tiene una valor de entrada en formato str, el cual corresponde al número binario
    '''
    resultado = 0
    for bit in binario:
        resultado = resultado * 2 + int(bit)
    return resultado

binario = "0101000011100001"
print("Resultado con myint:", myint(binario))
print("Resultado con int:", int(binario, 2))

Resultado con myint: 20705
Resultado con int: 20705


# 2.
Generar una función llamada `number64` para flotantes de precisión doble (64bits) donde a partir de un número binario encuentre el valor real. Compruebe su solución usando

```
number64("0100000000111011100100001111111111111111111111111111111111111111")
```

In [15]:
def number64(binario):

    # Verificamos que la cadena tenga exactamente 64 bits
    if len(binario) != 64:
        raise ValueError("La cadena debe tener 64 bits")

    # Primer bit: signo (0 = positivo, 1 = negativo)
    signo = int(binario[0])

    # Siguientes 11 bits: exponente
    exponente_bits = binario[1:12]

    # Últimos 52 bits: mantisa
    mantisa_bits = binario[12:]

    #Recorremos cada bit del exponente y calculamos su valor en base 10
    exponente = 0
    for i in range(len(exponente_bits)):
        bit = int(exponente_bits[i])  #Convertimos el carácter '0' o '1' a número entero
        potencia = len(exponente_bits) - 1 - i  #Calculamos la potencia de 2 correspondiente
        exponente += bit * (2 ** potencia)

    # Restamos el 1023 para obtener el exponente real
    exponente -= 1023

    # Cálculo de la mantisa
    # En IEEE 754 la mantisa se normaliza como 1.fracción, así que se inicia con 1.0
    mantisa = 1.0

    for i, bit in enumerate(mantisa_bits):
        # Cada bit de la mantisa representa una fracción binaria 2^-(i+1)
        mantisa += int(bit) * 2 ** -(i + 1)

    # Resultado final
    result = mantisa*(2**exponente)
    if signo == 1:
        result = -result
    return result

binario_64 = "0100000001011110110000000000000000000000000000000000000000000000"
print("Resultado:", number64(binario_64))

Resultado: 123.0


# 3. Aproximación de $\pi$

La serie de Maclaurin para la función tangente inversa converge en $-1 < x \leq 1$ y está dada por:

$$
\arctan x = \lim_{n \to \infty} P_n(x) = \lim_{n \to \infty} \sum_{i=1}^n (-1)^{i+1} \frac{x^{2i-1}}{2i-1}
$$

- **Aproximación básica**
    - Utilizando el hecho de que $\tan(\pi/4) = 1$, implemente en Python una función que calcule $4P_n(1)$ y determina el número mínimo de términos $n$ necesarios para que $|4P_n(1) - \pi| < 10^{-3}$. Compare su resultado con el valor de $\pi$ de la librería math.
    - Requiriendo que el valor de $\pi$ esté dentro de un error de $10^{-4}$. ¿Cuántos términos de la serie se necesitaría sumar para obtener esta precisión? Implemente una solución en Python para encontrar este valor.
- **Mejora de la convergencia**
    El método anterior puede mejorarse significativamente usando la identidad:
    $$
    \frac{\pi}{4} = \arctan\left(\frac{1}{2}\right) + \arctan\left(\frac{1}{3}\right)
    $$
    - Implemente en Python una función que evalúe la serie para $\arctan(1/2)$ y $\arctan(1/3)$ por separado y luego las sume. Determine el número mínimo de términos necesarios para cada serie (pueden ser diferentes) para aproximar $\pi$ con un error menor a $10^{-3}$ y $10^{-6}$.

- **Método de alta precisión**
    Para obtener una convergencia aún más rápida, usamos la identidad:
    $$
    \frac{\pi}{4} = 4\arctan\left(\frac{1}{5}\right) - \arctan\left(\frac{1}{239}\right)
    $$
    Desarrolle un programa en Python que implemente esta fórmula y determine:
    - El número mínimo de términos necesarios en cada serie para aproximar $\pi$ con error menor a $10^{-3}$ y $10^{-6}$.
    - Compare el rendimiento (número de términos requeridos) con los métodos anteriores.



In [16]:
# Aproximacion Basica

def metodo_basico(tol):
  serie = 0
  n = 1 # Contador de iteraciones

  while True:
    termino = ((-1)**(n+1)) / (2 * n - 1) # Se calcula el termino n-esimo de la suseción, con x = 1
    serie += termino
    pi_approx = 4 * serie                 # Se multiplica por 4 la serie para calcular el valor aproximado de pi
    error = abs(pi_approx - mt.pi)

    if error < tol:
      return pi_approx , error , n

    n +=1

# Inciso 1
pi_aprox, error, n = metodo_basico(1e-3)
print(f"Aproximación de pi: {pi_aprox}")
print(f"Error absoluto: {error}")
print(f"Número mínimo de términos necesarios: {n}")
print()
# Inciso 2
pi_aprox, error, n = metodo_basico(1e-4)
print(f"Aproximación de pi: {pi_aprox}")
print(f"Error absoluto: {error}")
print(f"Número mínimo de términos necesarios: {n}")

Aproximación de pi: 3.140592653839794
Error absoluto: 0.000999999749998981
Número mínimo de términos necesarios: 1000

Aproximación de pi: 3.1414926535900345
Error absoluto: 9.99999997586265e-05
Número mínimo de términos necesarios: 10000


In [17]:
# Mejora de la convergencia

def arctang(x, tol):
  '''
  Calcula la aproximacion de arctang con su serie de Mclaurin
  '''

  serie = 0
  n = 1 # Contador de iteraciones
  while True:
    termino = ((-1)**(n + 1)) * (x**(2*n - 1)) / (2*n - 1)
    serie += termino

    if abs(termino) < tol:
      return serie, n
    n += 1



def mejorar_convergencia_pi(tol):
  arctang_1, n1 = arctang(1/2, tol) # Evaluamos arctang(1/2)
  arctang_2, n2 = arctang(1/3, tol) # Evaluamos arctang(1/3)

  pi_aprox = 4 * (arctang_1 + arctang_2)
  error = abs(pi_aprox - mt.pi)

  return pi_aprox, error, n1, n2

# Error de pi menor a 1e-3
pi_aprox, error, n1, n2 = mejorar_convergencia_pi(tol=1e-3)
print(f"Aproximación de pi: {pi_aprox}")
print(f"Error absoluto: {error}")
print(f'Términos en arctan(1/2): {n1}')
print(f'Términos en arctan(1/3): {n2}')

print()

# Error de pi menor a 1e-6
pi_aprox, error, n1, n2 = mejorar_convergencia_pi(tol=1e-6)
print(f"Aproximación de pi: {pi_aprox}")
print(f"Error absoluto: {error}")
print(f'Términos en arctan(1/2): {n1}')
print(f'Términos en arctan(1/3): {n2}')

Aproximación de pi: 3.1419799015285124
Error absoluto: 0.00038724793871924845
Términos en arctan(1/2): 5
Términos en arctan(1/3): 3

Aproximación de pi: 3.1415928051045654
Error absoluto: 1.5151477228414478e-07
Términos en arctan(1/2): 9
Términos en arctan(1/3): 6


In [18]:
# Alta precision

def alta_precision(tol):
  arctang_1, n1 = arctang(1/5, tol) # Evaluamos arctang(1/5)
  arctang_2, n2 = arctang(1/239, tol) # Evaluamos arctang(1/239)

  pi_aprox = 4 * (4 * arctang_1 - arctang_2)
  error = abs(pi_aprox - mt.pi)

  return pi_aprox, error, n1, n2

# Error de pi menor a 1e-3
pi_aprox, error, n1, n2 = alta_precision(tol=1e-3)
print(f"Aproximación de pi: {pi_aprox}")
print(f"Error absoluto: {error}")
print(f'Términos en arctan(1/5): {n1}')
print(f'Términos en arctan(1/239): {n2}')

print()

# Error de pi menor a 1e-6
pi_aprox, error, n1, n2 = alta_precision(tol=1e-6)
print(f"Aproximación de pi: {pi_aprox}")
print(f"Error absoluto: {error}")
print(f'Términos en arctan(1/5): {n1}')
print(f'Términos en arctan(1/239): {n2}')


# Este ultimo metodo utiliza meenor cantidad de termninos que los dos anteriores
# Ya que el metodo basico usaba hasta 10000 terminos para tener un error menor a 1e-4,
# Mientras que el segundo metodo llego a usar en total 15 terminos para un error menor a 1e-6

Aproximación de pi: 3.1416210293260605
Error absoluto: 2.8375736267349794e-05
Términos en arctan(1/5): 3
Términos en arctan(1/239): 2

Aproximación de pi: 3.141592682405425
Error absoluto: 2.8815632102663358e-08
Términos en arctan(1/5): 5
Términos en arctan(1/239): 2


# 4. Serie Exponencial
Considere la serie para $e^{-x}$

$$
e^{-x}=\sum_{n=0}^{N}  (-1)^n \frac{x^{n}}{n!}
$$

- Calcula la serie para $x \le 1$ y compárela con la función incorporada `np.exp(x)` (asuma que la función exponencial incorporada es exacta). Elegir un $N$ para el cual el siguiente término en la serie no sea más que $10^{-7}$ de la suma hasta ese punto.

$$
\left| \frac{(-x)^{N+1}}{(N+1)!} \right | \le \left| 10^{-7} \sum_{N=0}^{N} \frac{(-x)^{n}}{n!} \right|
$$

- Examine los términos de la serie para $x\approx 10$ y observa las cancelaciones sustractivas significativas que ocurren cuando términos grandes se suman para dar respuestas pequeñas. En particular, imprime la cancelación casi perfecta en $n \approx x − 1$.

- Compruebe si se obtiene una mejor precisión siendo ingenioso y usando $e^{−x} = \frac{1}{e^x}$ para valores grandes de $x$. Esto elimina la cancelación sustractiva, pero no elimina todos los errores de redondeo.


- Incrementando progresivamente $x$ de 1 a 10, y luego de 10 a 100, use el programa para determinar experimentalmente cuándo la serie comienza a perder precisión, y cuándo la serie ya no converge.


- Realice una serie de gráficos del error versus $N$ para diferentes valores de $x$.

In [19]:
# Inciso 1:

def exp_xnegativo(x , tol = 1e-7):
  '''
  Calcula una aproximación de e^(-x) usando la serie de Maclaurin
  y determina el valor mínimo de N t.q el siguiente término
  es menor que una tolerancia.
  '''

  serie = 0
  n = 0
  sucesion = [] # Creo una lista para cuardar los terminos de la sucesion, esto nos servira para el inciso 2

  while True:
    a_n = (-x)**n / mt.factorial(n) # Calculamos el termino a_n de la sucesion
    sucesion.append(a_n)
    serie += a_n
    termino_n_mas_1 = (-x)**(n+1) / mt.factorial(n+1) # Calculamos el termino a_(n+1)

    if abs(termino_n_mas_1) <= abs(tol * serie): # Evaluamos si el N encontrado cumple con la condicion
      return serie , n , sucesion

    # Se le suma 1 a n, para seguir con la sumatoria en la siguiente iteracion
    n +=1

# Comparacion con np.exp

sumatoria , N , sucesion = exp_xnegativo(0.5)
print(f'Aproximación: {sumatoria}')
print(f'Valor de N: {N}')
print(f'np.exp(-0.5): {np.exp(-0.5)}')
print(f'Error relativo: {abs((sumatoria - np.exp(-0.5))/np.exp(-0.5))}')

Aproximación: 0.6065306648375496
Valor de N: 8
np.exp(-0.5): 0.6065306597126334
Error relativo: 8.449558280442002e-09


In [20]:
# Inciso 2:

sumatoria , N ,sucesion = exp_xnegativo(10)
print(f'Aproximación: {sumatoria}')
print(f'Valor de N: {N}')

for n, termino in enumerate(sucesion[8:12]):
    print(f"n = {n+8}, término = {termino}")

Aproximación: 4.539992793609265e-05
Valor de N: 45
n = 8, término = 2480.15873015873
n = 9, término = -2755.731922398589
n = 10, término = 2755.731922398589
n = 11, término = -2505.210838544172


In [21]:
# Inciso 3:

def exp_decreciente(x , tol = 1e-7):
  '''
  Calcula una aproximación de e^(-x) usando la serie de Maclaurin de e^x y luego e^(-x) = 1/e^x
  y determina el valor mínimo de N t.q el siguiente término es menor que una tolerancia.
  '''

  serie = 0
  n = 0

  while True:
    serie += (x)**n / mt.factorial(n)
    termino_n_mas_1 = (x)**(n+1) / mt.factorial(n+1) # Calculamos el termino a_(n+1)

    if abs(termino_n_mas_1) <= abs(tol * serie): # Evaluamos si el N encontrado cumple con la condicion
      return 1/serie , n

    # Se le suma 1 a n, para seguir con la sumatoria en la siguiente iteracion
    n +=1

# Comparacion con np.exp

sumatoria1 , N1 , sucesion = exp_xnegativo(30)
sumatoria2 , N2  = exp_decreciente(30)


print(f'Aproximación con el primer metodo: {sumatoria1}')
print(f'Aproximación con el segundo metodo: {sumatoria2}')
print(f'np.exp(30): {np.exp(-30)}')
print(f'Error relativo, primer metodo: {abs((sumatoria1 - np.exp(-30))/np.exp(-30))}')
print(f'Error relativo, segundo metodo: {abs((sumatoria2 - np.exp(-30))/np.exp(-30))}')

# Notese que para x grande, el segundo metodo es mas preciso y tiene una error absoluto menor

Aproximación con el primer metodo: -8.553016807103633e-05
Aproximación con el segundo metodo: 9.357623909887172e-14
np.exp(30): 9.357622968840175e-14
Error relativo, primer metodo: 914015968.0446448
Error relativo, segundo metodo: 1.0056474811858508e-07


In [22]:
# Inciso 4

for x_val in range(1,11):

  sumatoria1 , N1 , sucesion = exp_xnegativo(x_val)
  sumatoria2 , N2  = exp_decreciente(x_val)

  print(f'x = -{x_val} , metodo 1: {sumatoria1}')
  print(f'x = -{x_val} , metodo 2: {sumatoria2}')
  print(f'np.exp(-{x_val}): {np.exp(-x_val)}')
  print(f'Error, metodo 1: {abs((sumatoria1 - np.exp(-x_val))/np.exp(-x_val))}')
  print(f'Error, metodo 2: {abs((sumatoria2 - np.exp(-x_val))/np.exp(-x_val))}')

x = -1 , metodo 1: 0.3678794642857144
x = -1 , metodo 2: 0.36787944486780905
np.exp(-1): 0.36787944117144233
Error, metodo 1: 6.283110569681271e-08
Error, metodo 2: 1.0047766466489264e-08
x = -2 , metodo 1: 0.13533528043580964
x = -2 , metodo 2: 0.13533528720270754
np.exp(-2): 0.1353352832366127
Error, metodo 1: 2.0695290944941182e-08
Error, metodo 2: 2.9305697277038223e-08
x = -3 , metodo 1: 0.04978706711474102
x = -3 , metodo 2: 0.04978706944564669
np.exp(-3): 0.049787068367863944
Error, metodo 1: 2.5169646776796257e-08
Error, metodo 2: 2.1647845119790218e-08
x = -4 , metodo 1: 0.01831563849793654
x = -4 , metodo 2: 0.018315639833598484
np.exp(-4): 0.01831563888873418
Error, metodo 1: 2.133682803424859e-08
Error, metodo 2: 5.158784307730659e-08
x = -5 , metodo 1: 0.006737946894946384
x = -5 , metodo 2: 0.00673794754548251
np.exp(-5): 0.006737946999085467
Error, metodo 1: 1.5455610260511264e-08
Error, metodo 2: 8.10925112851672e-08
x = -6 , metodo 1: 0.0024787523124019917
x = -6 , met

In [23]:
for x_val in range(10,101):

  sumatoria1 , N1 , sucesion = exp_xnegativo(x_val)
  sumatoria2 , N2  = exp_decreciente(x_val)

  print(f'x = -{x_val} , metodo 1: {sumatoria1}')
  print(f'x = -{x_val} , metodo 2: {sumatoria2}')
  print(f'np.exp(-{x_val}): {np.exp(-x_val)}')
  print(f'Error, metodo 1: {abs((sumatoria1 - np.exp(-x_val))/np.exp(-x_val))}')
  print(f'Error, metodo 2: {abs((sumatoria2 - np.exp(-x_val))/np.exp(-x_val))}')

# Se puede apreciar que a partir de e^-29, el error absoluto del primer metodo supera 10e-6,
# minetras que el segundo metodo se mantiene estable mientras x se vuelve grande
# y no se tiene mucha diferernica con el valor exacto

x = -10 , metodo 1: 4.539992793609265e-05
x = -10 , metodo 2: 4.539993338712231e-05
np.exp(-10): 4.5399929762484854e-05
Error, metodo 1: 4.0228965480465895e-08
Error, metodo 2: 7.983795293024709e-08
x = -11 , metodo 1: 1.670170175035611e-05
x = -11 , metodo 2: 1.6701701886428723e-05
np.exp(-11): 1.670170079024566e-05
Error, metodo 1: 5.748578913371435e-08
Error, metodo 2: 6.563302011310534e-08
x = -12 , metodo 1: 6.144212973303477e-06
x = -12 , metodo 2: 6.144212674977195e-06
np.exp(-12): 6.14421235332821e-06
Error, metodo 1: 1.0090394536585432e-07
Error, metodo 2: 5.234991353228919e-08
x = -13 , metodo 1: 2.260327894757601e-06
x = -13 , metodo 2: 2.2603296728259734e-06
np.exp(-13): 2.2603294069810542e-06
Error, metodo 1: 6.690279074055545e-07
Error, metodo 2: 1.1761335244493259e-07
x = -14 , metodo 1: 8.315326009473704e-07
x = -14 , metodo 2: 8.315287922247079e-07
np.exp(-14): 8.315287191035679e-07
Error, metodo 1: 4.668321987431253e-06
Error, metodo 2: 8.793579629698855e-08
x = -15 ,

# 5.

Supongamos que tenemos una función $f(x)$ y queremos calcular su derivada en un punto $x$. Podemos hacerlo manualmente si conocemos la forma matemática de la función, o podemos hacerlo computacionalmente usando la definición de derivada:

$$
\frac{df}{dx} = \lim_{\delta \to 0} \frac{f(x + \delta) - f(x)}{\delta}.
$$

En el computador no podemos tomar el límite cuando $\delta$ tiende a cero, pero podemos obtener una aproximación razonable usando valores pequeños de $\delta$.

- Escriba un programa que:
    - Defina una función $f(x)$ que retorne el valor $x(x-1)$
    - Calcule la derivada de la función en $x = 1$ usando la fórmula anterior con $\delta = 10^{-2}$
    - Compare este resultado con el valor exacto obtenido analíticamente
    
    Los resultados no coincidirán exactamente. ¿Por qué?
    
- Repita el cálculo para $\delta = 10^{-4}, 10^{-6}, 10^{-8}, 10^{-10}, 10^{-12}$, y $10^{-14}$. Observará que la precisión mejora inicialmente al disminuir $\delta$, pero luego empeora. Explique este comportamiento.


In [24]:
def derivada(f , x , delta):
  df = (f(x + delta) - f(x))/delta
  return df

f = lambda x: x*(x-1)
df = derivada(f , 1 , 1e-2)
print(f'Delta=1e-2: {df}')
# Analiticamente tenemos que f'(x) = 2x-1, por lo cual, f'(1)=1
# Se puede observar que el valor obtenido con la funcion "derivada" difiere en un 0.01 con el valor real de la derivadad
# Esto puede deverse justamente valor delta=0.01

df2 = derivada(f , 1 , 1e-4)
print(f'Delta=1e-4: {df2}')
df3 = derivada(f , 1 , 1e-6)
print(f'Delta=1e-6: {df3}')
df4 = derivada(f , 1 , 1e-8)
print(f'Delta=1e-8: {df4}')
df5 = derivada(f , 1 , 1e-10)
print(f'Delta=1e-10: {df5}')
df6 = derivada(f , 1 , 1e-12)
print(f'Delta=1e-12 {df6}')
df7 = derivada(f , 1 , 1e-14)
print(f'Delta=1e-14: {df7}')

# Se puede apreciar que apartir de 10^-10, la precicion del valor de la derivada disminulle,
# esto puede deverse a que el valor de delta comienza a acercarse mas al cero de la maquina,
# generando un error en la operacion

Delta=1e-2: 1.010000000000001
Delta=1e-4: 1.0000999999998899
Delta=1e-6: 1.0000009999177333
Delta=1e-8: 1.0000000039225287
Delta=1e-10: 1.000000082840371
Delta=1e-12 1.0000889005833413
Delta=1e-14: 0.9992007221626509
